In [ ]:
import pandas as pd
import numpy as np
import tensorflow_probability

# from plotly.offline import init_notebook_mode
import plotly.graph_objects as go
import plotly.io as pio
from plotly.express import line
from src.em.mixture import DynamicMixture

pio.renderers.default = "notebook"

# Позволяет использовать измененные модули без перезагрузки ядра
%load_ext autoreload
%autoreload 2

# Extracting data

This is fresh data - 2020 year

I use specific date format for nice visualization. So first of all 
it's necessary to check if giving data has all important columns for
future work.

In [ ]:
from os.path import isfile

# If data don't contain my date format column
if not isfile("Data/2020_ydhm_id.csv"):
    data = pd.read_csv("Data/2020.csv", na_values=["99999.9", "9999.99"])

    def time_related_id(dataframe):
        from pandas import to_datetime

        # Converter: day number into specific date
        def day_to_date(day):
            date = to_datetime(day, format="%j")
            return date.strftime("%m-%d")

        dataframe["Date"] = dataframe["Day"].apply(day_to_date)

        # Converter for hours and minutes to proper representation
        def format_value(value):
            return f"{value:02}"

        # Construct date column with specific format
        def make_cell(row):
            return str(
                f"{row['Year']}"
                + f"-{row['Date']}"
                + f"T{format_value(row['Hour'])}"
                + f":{format_value(row['Minute'])}"
            )

        dataframe["ydhm_id"] = dataframe.apply(make_cell, axis=1)

        # Get rid of useless columns
        dataframe.drop(["Year", "Day", "Hour", "Minute", "Date"], axis=1, inplace=True)

    time_related_id(data)
    data[:525_600].to_csv("Data/2020_ydhm_id.csv", index=False)

else:
    data = pd.read_csv("Data/2020_ydhm_id.csv")

Expand data with component's increments

In [ ]:
def increm(arr):
    """
    Calculate increments of given array.
    First value is NaN by default
    """
    new_ar = [None]
    for i in range(1, len(arr)):
        inc = arr[i] - arr[i - 1]
        new_ar.append(inc)
    return new_ar


data["dBx"] = increm(data["Bx"].values)
data["dBy"] = increm(data["By"].values)
data["dBz"] = increm(data["Bz"].values)
data["dVx"] = increm(data["Vx"].values)
data["dVy"] = increm(data["Vy"].values)
data["dVz"] = increm(data["Vz"].values)

In [ ]:
data

But did we get all dates? The answer is yes.

In [ ]:
from datetime import datetime, timedelta

# Convert dates to temporal format. That enables dates subtruction
datetime_start = datetime.strptime("2023-01-01T00:00", "%Y-%m-%dT%H:%M")
datetime_end = datetime.strptime("2023-12-31T23:59", "%Y-%m-%dT%H:%M")

time_span = datetime_end - datetime_start
num_min = int(time_span.total_seconds() / 60.0) + 1
print(
    f"Number of minutes is {num_min}.\nIt is equal to {(time_span+timedelta(seconds=60))} hours."
)

Now we can start the research

# General overview of components

In [ ]:
# Extraction components names
comp_names = data.columns.values
comp_names = comp_names[~(comp_names == "ydhm_id")]
comp_names

## Visualization

There is no use of plotting all 500 000 points, so we'll limit observation by
one point per hour

In [ ]:
data_hourly = data[::60]
data_hourly

In [ ]:
from src.em.monitor import show_genral_info

for comp in comp_names:
    series = data_hourly[comp]
    fig = show_genral_info(
        series=series,
        add_title=f"Регулярность - один отсчет в час.\n Пропусков в исходных данных - {sum(pd.isna(data[comp].values))}",
        add_xaxis=data_hourly["ydhm_id"],
    )
    fig.update_layout(height=800)
    # fig.show()


del fig
del series

As we may notice amount of missing values for original components and 
their increments differs. This is a result of subtraction from `NaN` value
recorder no-`NaN` value. Take a look on rows 1, 2 and 3 for components 
__V__ and their increments __dV__.

In [ ]:
data.head()

## Histograms approximations

I've used generalized laplace distribution from 
[scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gennorm.html)
module and [article](https://www.mathnet.ru/links/611562f102eb3b9fd17ba6d59dc26199/ia230.pdf):

$$
f(x, \beta) = \frac{\beta}{2\Gamma (1 /\beta)} \cdot e^{-|x|^\beta},\newline 

\text{where } x \text{ is a real number }, \beta > 0, 
\Gamma \text{ is the gamma function.}
$$

The dVx, dVy, dVz data were preprocessed:
- Missing values were omitted 
- Outliers were cropped
- <u><b>Zero values were omitted</b></u>

Last assumption appeared because of the histogram view. There was enormous pick for zero value. Maybe some NASA's hardware errors

In [ ]:
from scipy.stats import gennorm as laplace


def filter_data(series, h0, h1):
    """Crop data based on given interval"""
    filtered_data = series[(series >= h0) & (series <= h1)]
    return filtered_data


def params_to_str(params):
    res = "beta= {0}, math.exp.= {1}, std.div.= {2}".format(
        *list(map(lambda x: round(x, 4), params))
    )
    return res


for comp_name in ["dVx", "dVy", "dVz"]:
    nice_series = data[comp_name].values
    nice_series = nice_series[~np.isnan(np.array(nice_series))]
    nice_series = nice_series[~(nice_series == 0.0)]
    nice_series = filter_data(nice_series, -25, 25)

    fig = go.Figure()
    fig.add_trace(
        go.Histogram(
            x=nice_series,
            nbinsx=200,
            histnorm="probability density",
            name="original data",
        )
    )

    distr_params = laplace.fit(nice_series)
    x = np.linspace(np.min(nice_series), np.max(nice_series), 4000)
    pdf_fitted = laplace.pdf(
        x,
        *distr_params,
    )

    fig.add_trace(go.Scatter(x=x, y=pdf_fitted, mode="lines", name="gen. Laplace"))
    fig.update_layout(
        title=dict(
            text=f"<b>{comp_name}</b> \t" + params_to_str(distr_params),
            font=dict(size=26),
        )
    )
    # fig.show()

# Mixtures of normal lows and coefficients of stochastic differential equations    

Assume that each time series is a mixture of __3__ normal lows.
Plots below represents reconstruction of stochastic coefficients for process:
$$ \delta X(t) = a(t) \delta t + b(t) \delta W ,$$ 
where X(t) stands for each ($B, dB, V, dV$) components projections.

Reconstruction was provided by EM-algorithm with Kolmogorov-Smirnov test, a.k.a.
"EM without dying variances" with sliding window's. Last one was next
parameters: step is __1 minute__, length is __3 days__.


**Assumption**: I'll drop out all `nan` values out of data.

## Mixtures of normal lows

In [ ]:
TIME = data["ydhm_id"].values
SERIES = dict()  # Container for components values on chosen time span
COLOR = dict(x="#1f77b4", y="#ff7f0e", z="#2ca02c")

for i, comp in enumerate(comp_names):
    time_series = data[comp].values.copy()
    SERIES[comp] = time_series  # Saving component values

SERIES.keys()

In [ ]:
window_size = 60 * 24 * 3 - 20  # 4320 minutes = 3 days

current_series = SERIES["dBx"]
mixt = DynamicMixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(window_size, 120),
)

mixt.rewrite_as_normal_human_this_initialization(
    random_seed=42, avr=np.mean(current_series)
)
mixt.predict_light(data=current_series[~np.isnan(current_series)])

In [ ]:
import pickle

with open("dBx_coeffs.pickle", "wb") as file:
    pickle.dump(mixt.process_coefs, file)

In [ ]:
with open("dBx_coeffs.pickle", "rb") as file:
    a = pickle.load(file)
a

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

windows = sliding_window_view(current_series[~np.isnan(current_series)], 4320)[::120]
comparison = dict(mean=list(map(np.mean, windows)), a=mixt.process_coefs["a"])

In [ ]:
line(comparison)

In [ ]:
mixt.show_parameters()

In [ ]:
import pickle

pickle.dump(mixt.reconstruct_process_coef(), "dBx")

In [ ]:
print(mixt)

In [ ]:
mixt.save(file_name="dBx_4300_60", path="Data/MyMixtures")

In [ ]:
data["dBx"]

In [ ]:
print(mixt)

In [ ]:
test: DynamicMixture = DynamicMixture.load("Data/MyMixtures/dBx_4300_60.pickle")

In [ ]:
a, b = mixt.reconstruct_process_coef()
coefs = dict(a=a, b_square=b, b=np.sqrt(b))
coefs_date = data["ydhm_id"][~np.isnan(current_series)][
    window_size // 2 : -window_size // 2 : 10
]
df_coefs = pd.DataFrame(coefs)

In [ ]:
df_coefs.corr(method="pearson")

In [ ]:
line(
    data_frame=df_coefs,
    x=data["ydhm_id"][::10][: len(a)],
    y=["a", "b", "b_square"],
    title="Process coefficients for mixture dBx: 3 components, 4300 window width and 10 min step",
).show()

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title="Correlation between a(t) and b^2(t) with window size 1 day (1440 min) and step 1 hour (60 min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 7  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title="Correlation between a(t) and b^2(t) with window size 1 week (10080 min) and step 1 hour (60 min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 7 * 4  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 1 month ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min).",
)

In [ ]:
shift_corr = 6 * 12  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 12 hour ({shift_corr*10} min).",
)

In [ ]:
def smooth(data, wind_size=20):
    from numpy.lib.stride_tricks import sliding_window_view
    from numpy import mean

    windows = sliding_window_view(data, wind_size)
    smoothed = []
    for wind in windows:
        smoothed.append(mean(wind))
    return smoothed

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

x_date = coefs_date[
    (window_corr_size + 24 * shift_corr) // 2 : -(window_corr_size + 24 * shift_corr)
    // 2
    + 7 : shift_corr
]
line(
    x=x_date,
    y=smooth(dynamic_corr["correlation"], 24),
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min). Smoothed by 1 day (24 counts)",
)

In [ ]:
window_size = 60 * 24 * 3  # 4320 minutes = 3 days

current_series = SERIES["dBy"]
mixt = DynamicMixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(window_size, 10),
)

mixt.rewrite_as_normal_human_this_initialization(
    random_seed=42, avr=np.mean(current_series)
)
mixt.predict_light(data=current_series[~np.isnan(current_series)])

In [ ]:
a, b = mixt.reconstruct_process_coef()
coefs = dict(a=a, b_square=b, b=np.sqrt(b))
coefs_date = data["ydhm_id"][~np.isnan(current_series)][
    window_size // 2 : -window_size // 2 : 10
]
df_coefs = pd.DataFrame(coefs)

In [ ]:
df_coefs.corr(method="pearson")

In [ ]:
line(
    data_frame=df_coefs,
    x=data["ydhm_id"][::10][: len(a)],
    y=["a", "b", "b_square"],
    title="Process coefficients for mixture <b>dBy</b>: 3 components, 4300 window width and 10 min step",
).show()

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title="Correlation between a(t) and b^2(t) with window size 1 day (1440 min) and step 1 hour (60 min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 7  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title="Correlation between a(t) and b^2(t) with window size 1 week (10080 min) and step 1 hour (60 min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 7 * 4  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 1 month ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

x_date = coefs_date[
    (window_corr_size + 24 * shift_corr) // 2 : -(window_corr_size + 24 * shift_corr)
    // 2
    + 7 : shift_corr
]
line(
    x=x_date,
    y=smooth(dynamic_corr["correlation"], 24),
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min). Smoothed by 1 day (24 counts)",
)

In [ ]:
window_size = 60 * 24 * 3  # 4320 minutes = 3 days

current_series = SERIES["dBz"]
mixt = DynamicMixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(window_size, 10),
)

mixt.rewrite_as_normal_human_this_initialization(
    random_seed=42, avr=np.mean(current_series)
)
mixt.predict_light(data=current_series[~np.isnan(current_series)])

In [ ]:
a, b = mixt.reconstruct_process_coef()
coefs = dict(a=a, b_square=b, b=np.sqrt(b))
coefs_date = data["ydhm_id"][~np.isnan(current_series)][
    window_size // 2 : -window_size // 2 : 10
]
df_coefs = pd.DataFrame(coefs)

In [ ]:
df_coefs.corr(method="pearson")

In [ ]:
line(
    data_frame=df_coefs,
    x=data["ydhm_id"][::10][: len(a)],
    y=["a", "b", "b_square"],
    title="Process coefficients for mixture <b>dBy</b>: 3 components, 4300 window width and 10 min step",
).show()

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title="Correlation between a(t) and b^2(t) with window size 1 day (1440 min) and step 1 hour (60 min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 7  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title="Correlation between a(t) and b^2(t) with window size 1 week (10080 min) and step 1 hour (60 min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 7 * 4  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 1 month ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

line(
    x=coefs_date[window_corr_size // 2 : -window_corr_size // 2 : shift_corr],
    y=dynamic_corr["correlation"],
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min).",
)

In [ ]:
shift_corr = 6  # in ten's of minutes (check step in mixture)
window_corr_size = 6 * 24 * 3  # in ten's of minutes (check step in mixture)

a_t = sliding_window_view(coefs["a"], window_shape=window_corr_size)[::shift_corr]

b_t = sliding_window_view(coefs["b_square"], window_shape=window_corr_size)[
    ::shift_corr
]
dynamic_corr = {"correlation": []}
for a, b in zip(a_t, b_t):
    dynamic_corr["correlation"].append(np.corrcoef(a, b)[0, 1])

x_date = coefs_date[
    (window_corr_size + 24 * shift_corr) // 2 : -(window_corr_size + 24 * shift_corr)
    // 2
    + 7 : shift_corr
]
line(
    x=x_date,
    y=smooth(dynamic_corr["correlation"], 24),
    title=f"Correlation between a(t) and b^2(t) with window size 3 days ({window_corr_size*10} min) and step 1 hour ({shift_corr*10} min). Smoothed by 1 day (24 counts)",
)